<a href="https://colab.research.google.com/github/amannarsaria4/mc_option_pricing/blob/main/MC_Option_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install ipympl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

from math import erf, sqrt
from scipy.special import erf

why we need MC simulation
  
analytical solutions make
- distribution assumption
- constant vol


##Simulate Paths

# Constant Volatility





## Naive MC simulations

In [4]:
def sim_gbm_paths(S_0, sigma, r, T, steps, n_paths, seed = 42):

  """
  Simulate a geometric brownian motion path for given n_steps and paths
  """

  rng = np.random.default_rng(seed)
  dt = T/steps
  n_half = (n_paths + 1)//2
  Z = rng.standard_normal((n_half, steps))
  Z = np.vstack([Z, -Z])[:n_paths,:]

  drift = (r - 0.5 * sigma**2) * dt
  vol = sigma * np.sqrt(dt)

  logS = np.empty((n_paths, steps + 1), dtype = float)
  logS[:,0]  = np.log(S_0)
  logS[:, 1:] = logS[:, [0]] + np.cumsum(drift + vol * Z, axis =1)

  return np.exp(logS)

In [5]:
def price_DKI_put_naive(S_0, K, B, sigma, r,T, n_steps, paths, seed = 42):

  S = sim_gbm_paths(S_0, sigma, r, T, n_steps, paths, seed)
  S_T = S[:,-1]

  hit = (S.min(axis=1) <= B)
  final_payoff = hit * np.maximum(K - S_T, 0.0)
  discounted_payoff = np.exp(-r * T) * final_payoff

  price_DKI_naive = discounted_payoff.mean()
  se_DKI_naive = discounted_payoff.std(ddof = 1) / np.sqrt(paths)

  return price_DKI_naive, se_DKI_naive


In [6]:
price_DKI = price_DKI_put_naive(100, 100, 70, 0.25,0.03,1,1000,100_000,42)
price_DKI

(np.float64(4.323790457422138), np.float64(0.03471912198407055))

## MC simulation with brownian bridge


In [31]:
def price_DKI_put_bridge(S_0, K, B, sigma, r,T, steps, n_paths, seed = 42):

  rng = np.random.default_rng(seed)
  dt = T / steps
  drift = (r - 0.5 * sigma**2) * dt
  vol = sigma * np.sqrt(dt)

  n_half = (n_paths + 1)//2
  Z = rng.standard_normal((n_half, steps))
  Z = np.vstack([Z, -Z])[:n_paths,:]

  logS = np.empty((n_paths, steps + 1), dtype = float)

  logS[:,0] = np.log(S_0)
  logS[:, 1:] = logS[:, [0]] + np.cumsum(drift + vol * Z, axis =1)

  #barrier in log space
  b = np.log(B)

  knocked_in = np.zeros(n_paths, dtype = bool)

  #brownian bridge logic
  for i in range(steps):
    x = logS[:,i]
    y = logS[:,i+1]

    still_out = ~knocked_in
    if not np.any(still_out):
      break

    x_still_out = x[still_out]
    y_still_out = y[still_out]

    # immediate hit if either point in below the barrier
    immediate_hit = (np.minimum(x_still_out, y_still_out) <= b)

    #get idx of paths not hit
    both_above = ~immediate_hit

    p = np.zeros_like(x_still_out)

    if np.any(both_above):
      xa = x_still_out[both_above]
      ya = y_still_out[both_above]

      p[both_above] = np.exp(-2.0 * (xa-b) * (ya-b) / (sigma**2 * dt))

    p[immediate_hit] = 1

    u = rng.random(p.shape)
    step_hit = (u<p)

    # update knocked_in for those still_out
    idx = np.where(still_out)[0]
    knocked_in[idx[step_hit]] = True

  S_T = np.exp(logS[:,-1])
  final_payoff = knocked_in * np.maximum(K - S_T, 0.0)
  disc_payoff = np.exp(-r * T) * final_payoff

  price_DKI_bridge = disc_payoff.mean()
  se_DKI_bridge = disc_payoff.std(ddof = 1) / np.sqrt(n_paths)

  return price_DKI_bridge, se_DKI_bridge

In [32]:
price_DKI_put_bridge = price_DKI_put_bridge(100, 100, 70, 0.25,0.03,1,1000,100_000,42)
price_DKI_put_bridge

(np.float64(4.423789952133973), np.float64(0.03491661506920041))

# Local Vol Model

- what is local vol ( emphasis on this)
-   why constant vol sucks
-   we need to fit market smile/curve and vol depends on strike and maturity

- assumptions for dupire

- how can we use raw market quote and use sabr to find local vol suraface
    Market quotes
    ↓
Fit smooth implied vol surface
    ↓
Convert to call prices
    ↓
Apply Dupire
    ↓
Local vol surface
    ↓
Monte Carlo / PDE pricing






In [11]:
# we derive this using transforming normal to erf

def norm_cdf(x):
  x = np.asarray(x, dtype = float)
  return 0.5 * (1.0 + erf(x / sqrt(2.0)))

In [12]:
# simple Black-Scholes formula to price call options
# we use forward moneyness to avoid rate and dividend distortions

def black_scholes(S_0, K, T, r, q, sigma):
  K = np.asarray(K, dtype = float)
  sigma = np.asarray(sigma, dtype = float)

  if T <= 0:
    return np.maximum(S_0 - K, 0)

  disc_r = np.exp(-r * T)
  disc_q = np.exp(-q * T)

  vol_sqrt = np.maximum(sigma * np.sqrt(T), 1e-12)  #stability
  d1 = (np.log(S_0 / K) + (r - q + 0.5 * sigma ** 2) * T) / vol_sqrt
  d2 = d1 - vol_sqrt

  return disc_q * S_0 * norm_cdf(d1) - disc_r * K * norm_cdf(d2)

In [14]:
# this is the sample vol suraface that we choose

%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

S0 = 100.0
r = 0.03
q = 0.01

def F(T):
    return S0 * np.exp((r - q) * T)

def sigma_imp(K, T):
    return 0.2 + 0.1 * ((K / F(T) - 1.0) ** 2) + 0.05 * np.sqrt(T)

# ----- Grid -----
K = np.linspace(60, 140, 120)      # strikes
T = np.linspace(0.02, 2.0, 100)    # maturities in years
K_grid, T_grid = np.meshgrid(K, T)

VOL = sigma_imp(K_grid, T_grid)

# ----- 3D Plot -----
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.plot_surface(K_grid, T_grid, VOL)

ax.set_xlabel("Strike K")
ax.set_ylabel("Maturity T (years)")
ax.set_zlabel("Implied Vol σ_imp")
ax.set_title("3D Implied Volatility Surface")

plt.show()


ValueError: Key backend: 'module://ipympl.backend_nbagg' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template', 'inline']

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()

In [13]:
def implied_vol_suraface(S_0, K,T,r,q):
  K = np.asarray(K, dtype=float)
  S_0 = np.asarray(S_0, dtype=float)

  F = S_0 * np.exp((r-q)*T)
  m = (K / F) - 1.0

  level = 0.20
  smile = 0.10 * (m**2)

  term = 0.05 * np.sqrt(max(T, 1e-12))

  sigma = level + smile + term

  return np.clip(sigma, 0.05, 2.00)


In [14]:
# crux; the meat; the juice
# getting dupire local vol from implied vol

def complute_local_vol(S_0, r,q,T_grid, K_grid):
  T_grid = np.asarray(T_grid, dtype = float)
  K_grid = np.asarray(K_grid, dtype = float)

  nT, nK = len(T_grid), len(K_grid)

  # step 1: compute call prices
  C = np.empty((nT, nK), dtype = float)
  for i, T in enumerate(T_grid):
    iv = implied_vol_suraface(S_0, K_grid, T, r,q)
    C[i,:] = black_scholes(S_0, K_grid, T, r, q, iv)


  # step 2(a) : compute first order differences in T
  C_T = np.empty_like(C)
  for i in range(nT):
      if i == 0:
          dT = T_grid[i + 1] - T_grid[i]
          C_T[i, :] = (C[i + 1, :] - C[i, :]) / dT
      elif i == nT - 1:
          dT = T_grid[i] - T_grid[i - 1]
          C_T[i, :] = (C[i, :] - C[i - 1, :]) / dT
      else:
          dT = T_grid[i + 1] - T_grid[i - 1]
          C_T[i, :] = (C[i + 1, :] - C[i - 1, :]) / dT

  # step 2(b) : compute first and second order differences in K
  C_K = np.empty_like(C)
  C_KK = np.empty_like(C)

  for j in range(nK):
    if j == 0:
        dK = K_grid[j + 1] - K_grid[j]
        C_K[:, j] = (C[:, j + 1] - C[:, j]) / dK
        dKc = 0.5 * (K_grid[j + 2] - K_grid[j])
        C_KK[:, j] = (C[:, j] - 2 * C[:, j + 1] + C[:, j + 2]) / (dKc**2)
    elif j == nK - 1:
        dK = K_grid[j] - K_grid[j - 1]
        C_K[:, j] = (C[:, j] - C[:, j - 1]) / dK
        dKc = 0.5 * (K_grid[j] - K_grid[j - 2])
        C_KK[:, j] = (C[:, j] - 2 * C[:, j - 1] + C[:, j - 2]) / (dKc**2)
    else:
        dK = K_grid[j + 1] - K_grid[j - 1]
        C_K[:, j] = (C[:, j + 1] - C[:, j - 1]) / dK
        dKc = 0.5 * (K_grid[j + 1] - K_grid[j - 1])
        C_KK[:, j] = (C[:, j + 1] - 2 * C[:, j] + C[:, j - 1]) / (dKc**2)

  # step 3 : calculate dupire local vol
  K = K_grid[None, :]
  num = C_T + (r - q) * K * C_K + q * C
  denom = 0.5 * K**2 * C_KK
  denom = np.clip(np.abs(denom) < 1e-14, np.nan, denom)

  sigma2 = num/denom
  sigma_loc = np.sqrt(sigma2)

  # fill nans
  for i in range(nT):
    row = sigma_loc[i, :]
    if np.any(~np.isfinite(row)):
        valid = np.isfinite(row)
        if valid.any():
            idx_valid = np.where(valid)[0]
            bad = np.where(~valid)[0]
            for j in bad:
                row[j] = row[idx_valid[np.argmin(np.abs(idx_valid - j))]]
        else:
            row[:] = 0.20
    sigma_loc[i, :] = row

  return C, sigma_loc


In [15]:
# Bi-linear interpolation

def make_sigma_loc_interpolator(T_grid, K_grid, sigma_loc_grid):

    T_grid = np.asarray(T_grid, dtype=float)
    K_grid = np.asarray(K_grid, dtype=float)
    sig = np.asarray(sigma_loc_grid, dtype=float)

    def sigma_loc(t, S):
        S = np.asarray(S, dtype=float)
        t = float(t)

        # clamp query
        t = float(np.clip(t, T_grid[0], T_grid[-1]))
        Kq = np.clip(S, K_grid[0], K_grid[-1])

        # T bracket
        i = np.searchsorted(T_grid, t) - 1
        i = int(np.clip(i, 0, len(T_grid) - 2))
        t0, t1 = T_grid[i], T_grid[i + 1]
        wt = 0.0 if t1 == t0 else (t - t0) / (t1 - t0)

        # K bracket (vectorized)
        j = np.searchsorted(K_grid, Kq) - 1
        j = np.clip(j, 0, len(K_grid) - 2).astype(int)
        K0 = K_grid[j]
        K1 = K_grid[j + 1]
        wK = np.where(K1 == K0, 0.0, (Kq - K0) / (K1 - K0))

        s00 = sig[i, j]
        s01 = sig[i, j + 1]
        s10 = sig[i + 1, j]
        s11 = sig[i + 1, j + 1]

        s0 = (1 - wK) * s00 + wK * s01
        s1 = (1 - wK) * s10 + wK * s11
        return (1 - wt) * s0 + wt * s1

    return sigma_loc

In [19]:
# log Euler time steps

def price_DKI_localvol_naive(S0, K, B, r, q, T, steps, n_paths, sigma_loc, seed=0):

    rng = np.random.default_rng(seed)
    dt = T / steps
    sqrt_dt = np.sqrt(dt)

    logS = np.full(n_paths, np.log(S0), dtype=float)
    hit = np.zeros(n_paths, dtype=bool)

    t = 0.0
    for _ in range(steps):
        S = np.exp(logS)
        sig = sigma_loc(t, S)

        Z = rng.standard_normal(n_paths)
        logS = logS + (r - q - 0.5 * sig**2) * dt + sig * sqrt_dt * Z
        S_next = np.exp(logS)

        # discrete monitoring
        hit |= (S_next <= B)
        t += dt

    ST = np.exp(logS)
    payoff = hit * np.maximum(K - ST, 0.0)

    disc = np.exp(-r * T)
    disc_payoff = disc * payoff
    price = disc_payoff.mean()
    se = disc_payoff.std(ddof=1) / np.sqrt(n_paths)
    return price, se

In [20]:
def price_DKI_localvol_bridge(S0, K, B, r, q, T, steps, n_paths, sigma_loc, seed=0):
    """
    "Brownian bridge" style monitoring under local vol (approx):
    - simulate endpoints with log-Euler (freeze local vol during step)
    - compute crossing probability assuming constant sigma = sigma_loc(t, S_t) within the step
    - sample hit event
    """
    rng = np.random.default_rng(seed)
    dt = T / steps
    sqrt_dt = np.sqrt(dt)
    b = np.log(B)

    logS = np.full(n_paths, np.log(S0), dtype=float)
    knocked_in = np.zeros(n_paths, dtype=bool)

    t = 0.0
    for _ in range(steps):
        S = np.exp(logS)
        sig = sigma_loc(t, S)   # frozen per-path for this step

        x = logS.copy()
        Z = rng.standard_normal(n_paths)
        logS = logS + (r - q - 0.5 * sig**2) * dt + sig * sqrt_dt * Z
        y = logS

        still_out = ~knocked_in
        if np.any(still_out):
            x0 = x[still_out]
            y0 = y[still_out]
            s0 = sig[still_out]

            # if either endpoint is below barrier => hit for sure
            immediate = (np.minimum(x0, y0) <= b)

            p = np.zeros_like(x0)
            p[immediate] = 1.0

            both_above = ~immediate
            if np.any(both_above):
                xa = x0[both_above]
                ya = y0[both_above]
                sa = s0[both_above]
                denom = np.maximum((sa**2) * dt, 1e-16)
                p_hit = np.exp(-2.0 * (xa - b) * (ya - b) / denom)
                p[both_above] = p_hit

            u = rng.random(p.shape)
            step_hit = (u < p)

            idx = np.where(still_out)[0]
            knocked_in[idx[step_hit]] = True

        t += dt

    ST = np.exp(logS)
    payoff = knocked_in * np.maximum(K - ST, 0.0)

    disc = np.exp(-r * T)
    disc_payoff = disc * payoff
    price = disc_payoff.mean()
    se = disc_payoff.std(ddof=1) / np.sqrt(n_paths)
    return price, se

In [21]:
if __name__ == "__main__":
    # Model / "market" params
    S0 = 100.0
    r = 0.03
    q = 0.00

    # Contract (Down-and-In Put)
    T = 1.0
    K = 100.0
    B = 80.0

    # Dupire grids (avoid T=0)
    T_grid = np.linspace(0.05, 2.0, 50)
    K_grid = np.linspace(50.0, 160.0, 250)

    # Build local vol surface from synthetic implied vol surface
    C_grid, sigma_loc_grid =complute_local_vol(
        S_0=S0, r=r, q=q, T_grid=T_grid, K_grid=K_grid)
    sigma_loc = make_sigma_loc_interpolator(T_grid, K_grid, sigma_loc_grid)

    # MC settings
    steps = 252
    n_paths = 100_000
    seed = 42

    p_naive, se_naive = price_DKI_localvol_naive(S0, K, B, r, q, T, steps, n_paths, sigma_loc, seed)
    p_bb, se_bb = price_DKI_localvol_bridge(S0, K, B, r, q, T, steps, n_paths, sigma_loc, seed)

    print("Down-and-In Put under Local Vol (synthetic IV -> Dupire -> MC)")
    print(f"Naive discrete monitoring: {p_naive:.6f}  (SE ~ {se_naive:.6f})")
    print(f"Frozen-localvol BB approx: {p_bb:.6f}  (SE ~ {se_bb:.6f})")

Down-and-In Put under Local Vol (synthetic IV -> Dupire -> MC)
Naive discrete monitoring: 4.552654  (SE ~ 0.029745)
Frozen-localvol BB approx: 4.705637  (SE ~ 0.029860)


# Stochastic Vol model

In [22]:
def DKI_discounted_payoff(S_T, K, knocked_in, r, T):
  payoff = knocked_in * np.maximum(K - S_T, 0.0)
  disc_payoff = np.exp(-r * T) *  payoff

  price = disc_payoff.mean()
  se = disc_payoff.std(ddof = 1) / np.sqrt(len(disc_payoff))

  return price, se

In [23]:
def simulate_heston_paths(S_0, r,q,T, n_steps, n_paths, v0, kappa, theta, xi, rho, seed = 42):

  rng = np.random.default_rng(seed)
  dt = T / n_steps
  sqrt_dt = np.sqrt(dt)

  logS = np.empty((n_paths, n_steps + 1), dtype = float)
  v = np.empty_like(logS)

  logS[:, 0] = np.log(S_0)
  v[:, 0] = v0

  for t in range(n_steps):
    Z1 = rng.standard_normal(n_paths)
    Z2 = rng.standard_normal(n_paths)

    W1 = Z1
    W2 = rho * Z1 + np.sqrt(max((1 - rho**2), 0.0)) * Z2

    v_pos = np.maximum(v[:, t], 0.0)

    v_next = v[:,t] + kappa * (theta - v_pos) * dt + xi * np.sqrt(v_pos) * sqrt_dt * W2
    v_next = np.maximum(v_next, 0.0)

    logS[:, t + 1] = logS[:, t] + (r - q - 0.5 * v_pos) * dt + np.sqrt(v_pos) * sqrt_dt * W1

    v[:, t+1] = v_next

  return logS, v

In [24]:
def price_DKI_heston_naive(S_0, K, B, r, q, T, n_steps, n_paths, v0, kappa, theta, xi, rho, seed = 42):

  logS, v = simulate_heston_paths(S_0, r, q, T, n_steps, n_paths, v0, kappa, theta, xi, rho)
  S = np.exp(logS)

  knocked_in = (S.min(axis = 1) <= B)

  S_T = S[:, -1]
  return DKI_discounted_payoff(S_T, K, knocked_in, r, T)

In [25]:
def price_DKI_heston_bridge(S_0, K, B, r, q, T, n_steps, n_paths, v0, kappa, theta, xi, rho, seed = 42):
  logS, v = simulate_heston_paths(S_0, r, q, T, n_steps, n_paths, v0, kappa, theta, xi, rho)

  dt = T / steps
  b = np.log(B)

  knocked_in = np.zeros(n_paths, dtype = bool)

  for t in range(steps):
    x = logS[:, t]
    y = logS[:, t + 1]

    v_pos = np.maximum(v[:, t], 0.0)  # we freeze start of step variance

    still_out = ~knocked_in
    if not np.any(still_out):
      break

    x0 = x[still_out]
    y0 = y[still_out]
    vv = np.maximum(v_pos[still_out], 1e-16)

    immediate = (np.minimum(x0, y0) <= b)

    p = np.zeros_like(x0)
    p[immediate] = 1.0

    both_above = ~immediate
    if np.any(both_above):
        xa = x0[both_above]
        ya = y0[both_above]
        va = vv[both_above]
        p_hit = np.exp(-2.0 * (xa - b) * (ya - b) / (va * dt))
        p[both_above] = np.clip(p_hit, 0.0, 1.0)

    u = np.random.default_rng(seed + 12345 + t).random(p.shape)
    step_hit = (u < p)

    idx = np.where(still_out)[0]
    knocked_in[idx[step_hit]] = True

  ST = np.exp(logS[:, -1])
  return DKI_discounted_payoff(ST, K, knocked_in, r, T)


In [26]:
S_0 = 100.0
K  = 100.0
B  = 80.0          # down barrier
T  = 1.0           # 1 year
r  = 0.03
q  = 0.00

steps   = 252
n_paths = 200_000
seed    = 42

v0    = 0.04
kappa = 2.0     # mean reversion speed
theta = 0.04
xi    = 0.60    # vol of vol
rho   = -0.70   # leverage effect

p_h_naive, se_h_naive = price_DKI_heston_naive(
        S_0=S_0, K=K, B=B, r=r, q=q, T=T, n_steps=steps, n_paths=n_paths, seed=seed,
        v0=v0, kappa=kappa, theta=theta, xi=xi, rho=rho
    )

p_h_bridge, se_h_bridge = price_DKI_heston_bridge(
    S_0=S_0, K=K, B=B, r=r, q=q, T=T, n_steps=steps, n_paths=n_paths, seed=seed,
    v0=v0, kappa=kappa, theta=theta, xi=xi, rho=rho)


print("HESTON results (Down-and-In Put):")
print(f"  Naive monitoring:  price={p_h_naive:.6f},  SE={se_h_naive:.6f}")
print(f"  Bridge monitoring: price={p_h_bridge:.6f}, SE={se_h_bridge:.6f}")
print()


HESTON results (Down-and-In Put):
  Naive monitoring:  price=5.007375,  SE=0.025930
  Bridge monitoring: price=5.112821, SE=0.025955



# Merton Jump Diffusion

In [27]:
def simulate_merton_paths(S_0, r, q, T, n_steps, n_paths, sigma, lam, muJ, sigJ, seed = 42):

  rng = np.random.default_rng(seed)
  dt = T / n_steps
  sqrt_dt = np.sqrt(dt)

  logS = np.empty((n_paths, n_steps +1), dtype = float)
  logS[:,0] = np.log(S_0)

  kappaJ = np.exp(muJ + 0.5 * sigJ ** 2) - 1.0

  drift = (r - q - lam*kappaJ - 0.5 * sigma**2)

  Nstep = np.empty((n_paths, n_steps), dtype = int)

  for t in range(steps):
    # count number of jumps
    N = rng.poisson(lam*dt, size = n_paths)
    Nstep[:, t] = N

    # calculate diffusion increment
    Z = rng.standard_normal(n_paths)
    diff = drift*dt + sigma*sqrt_dt*Z

    # sum of log-jumps
    jump_sum = np.zeros(n_paths, dtype = float)
    idx = np.where(N > 0)[0]
    if idx.size > 0:
      jump_sum[idx] = (N[idx] * muJ) + np.sqrt(N[idx]) * sigJ * rng.standard_normal(idx.size)

    logS[:, t+1] = logS[:, t] + diff + jump_sum

  return logS, Nstep


In [28]:
def price_dip_merton_naive_mc(S_0, K, B, r, q, T, n_steps, n_paths, sigma, lam, muJ, sigJ):
    logS, Nstep = simulate_merton_paths(S0, r, q, T, n_steps, n_paths, sigma, lam, muJ, sigJ)
    S = np.exp(logS)

    knocked_in = (S.min(axis=1) <= B)
    ST = S[:, -1]
    return DKI_discounted_payoff(ST, K, knocked_in, r, T)

In [29]:
S_0 = 100.0
K  = 100.0
B  = 80.0          # down barrier
T  = 1.0           # 1 year
r  = 0.03
q  = 0.00

steps   = 252
n_paths = 200_000
seed    = 42

v0    = 0.04
kappa = 2.0     # mean reversion speed
theta = 0.04
xi    = 0.60    # vol of vol
rho   = -0.70   # leverage effect

sigma = 0.20   # diffusion vol
lam   = 0.60   # jumps per year (Poisson intensity)
muJ   = -0.10  # mean log jump (downward bias)
sigJ  = 0.25   # jump size vol (log space)

p_m_naive, se_m_naive = price_dip_merton_naive_mc(
        S_0=S_0, K=K, B=B, r=r, q=q, T=T, n_steps=steps, n_paths=n_paths,
        sigma=sigma, lam=lam, muJ=muJ, sigJ=sigJ
  )


In [30]:
p_m_naive

np.float64(7.762570198162501)